# Aujourd'hui on roule sur les mecs de l'ENS


https://challengedata.ens.fr/en/challenge/39/prediction_of_transaction_claims_status.html

# Imports des librairies de bases

On ajoutera celles qui manquent au fur et à mesure de nos besoins

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import pandas as pd
import os, gc

# Définition de la seed pour le random

Très important pour qu'on voit les mêmes choses entre nos deux ordis

In [2]:
RANDOM_SEED = 42;
np.random.seed(RANDOM_SEED)

# Définition des paramètres pour Matplot

Rien de bien intéréssant

In [3]:
# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Set des variables globales

Attention, je n'utilise les variables globales pour la gestion des fichiers. Sinon, c'est mort

In [9]:
# Where to save the figures
PROJECT_ROOT_DIR = "."
DATA_PROCESSED = os.path.join(PROJECT_ROOT_DIR, "/data_processed")

# Fonction pour load les libraires

En vrai, on a juste besoin de pd.read_csv, mais c'était pour faire joli

In [10]:
def load_data(file,data_path=DATA_PROCESSED, sep=';'):
    csv_path = os.path.join(data_path, file)
    return pd.read_csv(csv_path, sep=';')

# On load les jeux de données

In [ ]:
pd.read_csv("./data_processed/train.csv)

In [12]:
TX_data = load_data(file = "train.csv");
TEST_DATA = load_data(file = "/test.csv");

FileNotFoundError: File b'/data_processed\\train.csv' does not exist

In [ ]:
TEST_DATA.head()

In [ ]:
TX_data.drop(['CARD_PAYMENT','COUPON_PAYMENT','RSP_PAYMENT','WALLET_PAYMENT', "BUYING_DATE"], axis = 1, inplace = True)
TEST_DATA.drop(['CARD_PAYMENT','COUPON_PAYMENT','RSP_PAYMENT','WALLET_PAYMENT', "BUYING_DATE"], axis = 1, inplace = True)

In [ ]:
TX_data.SHIPPING_MODE = TX_data.SHIPPING_MODE.astype('category')
TX_data.PRICECLUB_STATUS = TX_data.PRICECLUB_STATUS.astype('category')
TX_data.SELLER_COUNTRY = TX_data.SELLER_COUNTRY.astype('category')
TX_data.PRODUCT_TYPE = TX_data.PRODUCT_TYPE.astype('category')
TX_data.PRODUCT_FAMILY = TX_data.PRODUCT_FAMILY.astype('category')


cat_columns = TX_data.select_dtypes(['category']).columns
cat_columns
TX_data[cat_columns] = TX_data[cat_columns].apply(lambda x: x.cat.codes)

# Jointure entre les X et Y

In [ ]:
def datapreprocess(data):
    data=data.apply(pd.to_numeric, errors='ignore')
    
    # Y and X
    try :
        Y=data["CLAIM_TYPE"]
        X=data.drop("CLAIM_TYPE", axis=1,inplace=False)
    except:
        Y=0
        X=data
    # Exclude Objets
    X=X.select_dtypes(exclude=['object'])
    
    # Work on fare
    from sklearn.preprocessing import Imputer
    imp = Imputer(missing_values='NaN',strategy='median', axis=1)
    X=pd.DataFrame(imp.fit_transform(X),columns=X.columns.values)
 
    return X, Y

In [ ]:
X_train, Y_train = datapreprocess(TX_data)
TEST_DATA, _ = datapreprocess(TEST_DATA)

#del TX_data;
gc.collect()

# MODEL!

In [ ]:
from sklearn.feature_selection import SelectFromModel

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_jobs=-1, 
                             random_state=RANDOM_SEED)

sfm_clf = SelectFromModel(clf)

del clf

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb_clf=GradientBoostingClassifier(random_state=RANDOM_SEED)

In [ ]:
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler

clf = Pipeline([
    #('scaler', StandardScaler()), 
    #('norm', Normalizer()), 
    #('threshold', sel_vt), 
    #('SelectKBest', Chi_select), 
    #('reduce_dim', PCA()),
    #('feature_union', combined),
    ('feature_selection', sfm_clf),
    ('classification', gb_clf)
])

clf.fit(X_train, Y_train);

In [ ]:
RESULTS["CLAIM_TYPE"] = pd.DataFrame(clf.predict(TEST_DATA), columns=["Result"])

In [ ]:
filename = DATA_PROCESSED+"/submission_2.csv"

RESULTS.to_csv(filename, index=False, sep=";")

In [ ]:
TEST_DATA.info()

In [ ]:
rnd_clf.fit(X_train, Y_train);
y_pred_rf = rnd_clf.predict(X_test)
multiclass_roc_auc_score(Y_test, y_pred_rf)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
def plot_confusion_matrix(matrix):
    """If you prefer color and a colorbar"""
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111)
    cax = ax.matshow(matrix)
    fig.colorbar(cax)
    
conf_mx = confusion_matrix(Y_test, y_pred_rf)
plot_confusion_matrix(conf_mx)

In [ ]:
row_sums = conf_mx.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx / row_sums
plot_confusion_matrix(norm_conf_mx)

In [ ]:
from rakuten import rakuten_ROC_CV

rakuten_ROC_CV(rnd_clf, X_train, Y_train, _n_splits=10, _random_state=RANDOM_SEED)

In [ ]:

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

In [ ]:
from sklearn.pipeline import Pipeline
clf = Pipeline([
    #('scaler', StandardScaler()), 
    #('norm', Normalizer()), 
    #('threshold', sel_vt), 
    #('SelectKBest', Chi_select), 
    #('reduce_dim', PCA()),
    #('feature_union', combined),
    ('feature_selection', sfm_clf),
    ('classification', gss)
])



multiclass_roc_auc_score(Y_test, y_pred_rf)

In [ ]:
conf_mx = confusion_matrix(Y_test, y_pred_rf)
row_sums = conf_mx.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx / row_sums
plot_confusion_matrix(norm_conf_mx)

In [ ]:
from xgboost import XGBClassifier

In [ ]:
model = XGBClassifier()

In [ ]:
model.fit(X_train, Y_train)

In [ ]:
print(model)

In [ ]:
y_pred_xgb = model.predict(X_test)
multiclass_roc_auc_score(Y_test, y_pred_xgb)

In [ ]:
conf_mx = confusion_matrix(Y_test, y_pred_xgb)
row_sums = conf_mx.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx / row_sums
plot_confusion_matrix(norm_conf_mx)